In [1]:
import cv2
import numpy as np
import pandas as pd
import os
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow.keras as K
import tensorflow.keras.backend as Kback
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import tensorflow as tf

# from google.colab import drive
# drive.mount('/content/drive')

# Model Training

In [3]:
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = Kback.sum(Kback.round(Kback.clip(y_true * y_pred, 0, 1)))
    possible_positives = Kback.sum(Kback.round(Kback.clip(y_true, 0, 1)))
    predicted_positives = Kback.sum(Kback.round(Kback.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + Kback.epsilon())
    recall = true_positives / (possible_positives + Kback.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+Kback.epsilon())
    return f1_val

METRICS = [
      "accuracy",
      K.metrics.Precision(name='precision'),
      K.metrics.Recall(name='recall'),
      K.metrics.AUC(name='auc'),
      f1_score
]

In [4]:
input_layer = K.Input(shape=(160,160,3))
resnet_model = K.applications.inception_v3.InceptionV3(include_top = False, weights = "imagenet", input_tensor = input_layer)
for layer in resnet_model.layers:
    layer.trainable = False
for i, layer in enumerate(resnet_model.layers):
    print(i, layer.name, "-", layer.trainable)

87910968/87910968 [==============================] - 47s 1us/step
0 input_1 - False
1 conv2d - False
2 batch_normalization - False
3 activation - False
4 conv2d_1 - False
5 batch_normalization_1 - False
6 activation_1 - False
7 conv2d_2 - False
8 batch_normalization_2 - False
9 activation_2 - False
10 max_pooling2d - False
11 conv2d_3 - False
12 batch_normalization_3 - False
13 activation_3 - False
14 conv2d_4 - False
15 batch_normalization_4 - False
16 activation_4 - False
17 max_pooling2d_1 - False
18 conv2d_8 - False
19 batch_normalization_8 - False
20 activation_8 - False
21 conv2d_6 - False
22 conv2d_9 - False
23 batch_normalization_6 - False
24 batch_normalization_9 - False
25 activation_6 - False
26 activation_9 - False
27 average_pooling2d - False
28 conv2d_5 - False
29 conv2d_7 - False
30 conv2d_10 - False
31 conv2d_11 - False
32 batch_normalization_5 - False
33 batch_normalization_7 - False
34 batch_normalization_10 - False
35 batch_normalization_11 - False
36 activation_5 - 

In [5]:
def fft_2d(feature_map):
    feature_map = tf.cast(feature_map, tf.complex64)
    X1 = tf.signal.fft2d(feature_map)
    X1 = tf.abs(X1)
    return X1   

inputs = K.layers.Input(shape=(160,160,3), name="image") 
resnet50 = resnet_model(inputs)
fft_layer = K.layers.Lambda(lambda x: fft_2d(x))(resnet50)
fft_flat = K.layers.GlobalAveragePooling2D()(fft_layer)
output = K.layers.Dense(20, activation='softmax')(fft_flat)

model = K.Model(inputs=inputs, outputs=fft_flat)
optimizer = K.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
model.compile(loss=["categorical_crossentropy"], metrics=METRICS, optimizer = optimizer)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image (InputLayer)          [(None, 160, 160, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 lambda (Lambda)             (None, 3, 3, 2048)        0         
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
Total params: 21,802,784
Trainable params: 0
Non-trainable params: 21,802,784
_________________________________________________________________


C:\Users\SAPTARSHI PANI\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [6]:
import os
import random
import numpy as np

Y_train = []
X_train = []
for i in range(1,21):
  folder = "E:/data/train/"+str(i)+"/"
  for image in os.listdir(os.path.join(folder)):
    I = []
    img = cv2.imread(os.path.join(folder,image))
    img = cv2.resize(img, (160, 160))
    img = img/255
    I.append(img)
    I = np.asarray(I)
    X_train.append(np.asarray(model.predict(I)[0]))
    Y_train.append(i)

Y_train = np.asarray(Y_train)
X_train = np.asarray(X_train)

Y_test = []
X_test = []
for i in range(1,21):
  folder = "E:/data/test/"+str(i)+"/"
  for image in os.listdir(os.path.join(folder)):
    I = []
    img = cv2.imread(os.path.join(folder,image))
    img = cv2.resize(img, (160, 160))
    img = img/255
    I.append(img)
    I = np.asarray(I)
    X_test.append(np.asarray(model.predict(I)[0]))
    Y_test.append(i)

Y_test = np.asarray(Y_test)
X_test = np.asarray(X_test)

shuffler = np.random.permutation(len(X_train))
X_train = X_train[shuffler]
Y_train = Y_train[shuffler]




 

 





#Load model
# model_xgb_2 = xgb.Booster()
# model_xgb_2.load_model("model.json")

1/1 [==============================] - 0s 85ms/step


1/1 [==============================] - 0s 93ms/step


1/1 [==============================] - 0s 126ms/step


1/1 [==============================] - 0s 71ms/step


1/1 [==============================] - 0s 89ms/step


...

1/1 [==============================] - 0s 69ms/step


1/1 [==============================] - 0s 65ms/step


1/1 [==============================] - 0s 52ms/step


1/1 [==============================] - 0s 58ms/step


1/1 [==============================] - 0s 56ms/step


In [7]:
shuffler = np.random.permutation(len(X_test))
X_test = X_test[shuffler]
Y_test = Y_test[shuffler]

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y_train = le.fit_transform(Y_train)

In [9]:
#XGBoost as classifier using the features of CNN+FFT
import xgboost as xgb
my_model = xgb.XGBClassifier(n_estimators = 400, learning_rate = 0.1, max_depth = 3)
my_model.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=400, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [10]:
# Predicting the Test set results
y_pred = my_model.predict(X_test)

In [11]:
# Making the Confusion Matrix
cm = confusion_matrix(Y_test, y_pred)
disp = ConfusionMatrixDisplay(cm,display_labels=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])

In [12]:
Y_test = le.fit_transform(Y_test)

In [13]:
#Accuracy
from sklearn.metrics import accuracy_score
print("Testing accuracy:")
print(accuracy_score(Y_test, y_pred))

Testing accuracy:
0.7395


In [14]:
cm

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [80,  6,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  1,  0,
         0,  0, 12,  0,  0],
       [ 7, 73,  0,  0,  0,  0,  0,  0,  0,  0,  3,  0,  0,  0,  2,  0,
         0,  0, 15,  0,  0],
       [ 0,  0, 74,  0,  0,  0,  0, 17,  1,  1,  1,  2,  0,  1,  0,  0,
         0,  0,  1,  2,  0],
       [ 0,  0,  1, 65, 12,  3,  0,  0,  0,  2,  0,  0,  7,  0,  0,  1,
         0,  0,  0,  9,  0],
       [ 1,  0,  0,  6, 92,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0],
       [ 1,  1,  0,  0,  2, 93,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,
         1,  0,  1,  0,  0],
       [ 3,  1,  0,  0,  0,  0, 80,  0,  3,  0,  0,  1,  0,  0,  0,  8,
         0,  3,  1,  0,  0],
       [ 3,  0, 22,  0,  0,  0,  2, 59,  9,  1,  1,  2,  0,  0,  0,  0,
         1,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  1,  1, 70,  2,  1, 23,  0,  0,  0,  0,
         0,  0,  0,

In [15]:
#F1_score
from sklearn.metrics import f1_score
print("Testing F1-score")
print(f1_score(Y_test, y_pred, pos_label='positive', average='macro'))

Testing F1-score
0.7410602577291491


C:\Users\SAPTARSHI PANI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1379: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [16]:
#Precision
from sklearn.metrics import precision_score
print("Testing Precision:")
print(precision_score(Y_test, y_pred, pos_label='positive', average='macro'))
#Recall
from sklearn.metrics import recall_score
print("Testing Recall:")
print(recall_score(Y_test, y_pred, pos_label='positive', average='macro'))

Testing Precision:
0.7557219421500672
Testing Recall:
0.7394999999999999


C:\Users\SAPTARSHI PANI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1379: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
C:\Users\SAPTARSHI PANI\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1379: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [17]:
# save to JSON
my_model.save_model("model_3.json")
# save to text format
my_model.save_model("model_3.txt")

In [18]:
X_train.shape

(10000, 2048)